# Functionality: visualization of functional connectivity

<ul>
    <li>142 channels = 1-96 channels (M1) + 97-100(not used) + 101-132(ThaSMA) + 133-146 (GP/STN)</li>
    <li>load animetafolder/pinky_locThaSMA.csv for the details of ThaSMA </li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

from igraph.drawing.text import TextDrawer
import cairo

In [2]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual

importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## exp subfolders & code_corresfolder

In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder,correparentfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [18]:
# threshold lowweight
lowweight = 0.15

# frequency
freq = [17, 19]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
eventphase = 'return'

In [6]:
### text setup for brain areas ###
texts_area = dict()

texts_area['VPLo'] = [20, 40, 20]

texts_area['VLo'] = [20, 120, 20]

texts_area['VA'] = [20, 200, 20]

texts_area['SMA'] = [20, 320, 20]

texts_area['M1'] = [20, 500, 20]

texts_area['STN'] = [280, 140, 20]

texts_area['GP'] = [360, 250, 20]

In [7]:
areas_unwanted = ['lCd','rMC']

## input setup

In [8]:
inputfolder = os.path.join(correparentfolder, 'm1_SKT_calcciCOH_return_Fre17-19Hz')
filename = 'ciCOH_STK_return_freq_17_19.pickle'

file = os.path.join(inputfolder, filename)

## savefile setup

In [19]:
savefolder = corresfolder
savefile_prefix = 'ciCOH_' + eventphase + '_' + strfreq 

## extract ciCOH

In [10]:
# load dictionary ciCOH
ciCOH = pickle.load(open(file, "rb" ))

In [11]:
print(ciCOH.keys())

dict_keys(['normal', 'mild', 'chnInf_recchni', 'chnInf_matchni', 'chnInf_brainarea', 'chnInf_coord'])


In [12]:
# ciCOH
ciCOH_normal, ciCOH_mild = ciCOH['normal'], ciCOH['mild']

# channel information
chnInf_recchni, chnInf_matchni = ciCOH['chnInf_recchni'], ciCOH['chnInf_matchni']
chnInf_brainarea, chnInf_coord = ciCOH['chnInf_brainarea'], ciCOH['chnInf_coord']

## del data in unwanted areas

In [13]:
def unwanted_chandata_del(ciCOH, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord):
    
    chns_unwanted = []
    for area_unwanted in areas_unwanted:
        for i, area in enumerate(chnInf_brainarea):
            if area == area_unwanted:
                chns_unwanted.append(i)
    
    nchns = ciCOH.shape[0]
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 0)
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 1)
    
    
    chnInf_recchni = [i for j, i in enumerate(chnInf_recchni) if j not in chns_unwanted]
    chnInf_brainarea = [i for j, i in enumerate(chnInf_brainarea) if j not in chns_unwanted]
    chnInf_coord = [i for j, i in enumerate(chnInf_coord) if j not in chns_unwanted]

    return ciCOH, chnInf_recchni, chnInf_brainarea, chnInf_coord

In [14]:
ciCOH_normal, chnInf_recchni, chnInf_brainarea, chnInf_coord = \
        unwanted_chandata_del(ciCOH_normal, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord)
ciCOH_mild, chnInf_recchni, chnInf_brainarea, chnInf_coord = \
        unwanted_chandata_del(ciCOH_mild, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord)

## ciCOH Visual

In [15]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH Tensor for each trial (nchns, nchns, ntrials)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    weight = abs(ciCOH)

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    # set graph visualization style
    visual_style = graphVisual.graph_style(graph)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(tobesavedfile)
    
    return igplot

### normal

In [20]:
"""
    normal condition
"""


# save file config
savefilename = savefile_prefix+ '_lowweight' + str(lowweight) + '_normal' + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_normal, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)


# del 
#del ciCOH_normal_subchns, savefile, texts, title

/Users/linglingyang/miniconda3/envs/general/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


Figure saved to /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/SKT/m2_SKT_visualFC_return_Fre17-19Hz/ciCOH_return_freq17_19_lowweight0.15_normal.png


### mild

In [21]:
"""
    mild condition
"""

condition = 'mild'


# save file config
savefilename = savefile_prefix + '_lowweight' + str(lowweight) + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()

# visualize and save
igplot = ciCOH_visual_save(ciCOH_mild, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)

# del 
del savefile, texts

/Users/linglingyang/miniconda3/envs/general/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


Figure saved to /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/SKT/m2_SKT_visualFC_return_Fre17-19Hz/ciCOH_return_freq17_19_lowweight0.15_mild.png
